## Wind API

+ 对于季度需要看一下，尤其是跨季度到底得到了什么
+  

In [10]:
from WindPy import w
import numpy as np
import pandas as pd
import sys,os,copy
import requests, json
import logging
import datetime,openpyxl
from requests_toolbelt import MultipartEncoder
from logging.handlers import TimedRotatingFileHandler
from win32com import client as wc

root = 'C:\\Users\\13260\\Desktop\\API'

log = logging.getLogger(__name__)
log.setLevel(logging.INFO)

file_formatter = logging.Formatter('%(asctime)s - %(funcName)s - %(levelname)s - %(message)s')
file_handler = TimedRotatingFileHandler(filename=root+'\\运行情况.log',when='D',backupCount=30,encoding='utf-8')
file_handler.setFormatter(file_formatter)
file_handler.setLevel(logging.INFO)
log.addHandler(file_handler)


class Wind_API():
    # 数值对应的月份应该会给出字符对应的季度的数据
    # 示例：当前是1月，则应当查询上一年的第四季度，所以是 1:'04'
    month_season={1:'04',2:'04',3:'04',
                  4:'01',5:'01',6:'01',
                  7:'02',8:'02',9:'02',
                  10:'03',11:'03',12:'03',
                  '01':2,'02':5,'03':8,'04':11}
    def __init__(self,api_setting_path,output_folder):
        # 输入文件与输出文件夹
        self.api_setting_path = api_setting_path
        self.output_folder = output_folder
        
        # 获取时间（一般是指从当前日期往前查询）
        self.today = datetime.date.today()
        self.month_season = Wind_API.month_season        
        
        # 读入输入文件（配置文件）
        self.refresh_api_setting()
        self.season = copy.deepcopy(self.api_setting.loc[self.api_setting['频率']=='季度',:])
        self.month = copy.deepcopy(self.api_setting.loc[self.api_setting['频率']=='月度',:])
    
    def refresh_api_setting(self):
        self.api_setting = pd.read_excel(self.api_setting_path,sheet_name="wind API", dtype = {'地区' : str,'已处理季度' : str})
        self.api_setting.fillna(' ',inplace=True)

    def to_update(self):
        # 进行查询
        self.update_month()
        self.update_season()
    
    def update_month(self):
        """
        更新频率为月的数据，更新上个月和上上个月
        """
        # 更新上上个月
        # diff：当前月份与已处理月份的差  1：正常；2：上个月需要更新；3：上上个月也需要更新；>3：存在问题（直接更新上上个月
        diff = (self.today.month-self.month.loc[:,'已处理月份']).map(lambda x: x+12 if x<0 else x)
        to_update = self.month.loc[diff>=3,'代码']
        self.refresh_api_setting()
        if list(to_update):
            indicators = ','.join(list(to_update))
            # 回推两个月，找到两个月前的开始月份
            month_first = datetime.date(self.today.year, self.today.month, 1)
            for i in range(2):
                month_last =  month_first - datetime.timedelta(1)
                month_first = datetime.date(month_last.year, month_last.month, 1)
            log.info('开始更新%s数据（上上个月）'%month_first.strftime('%Y-%m'))
            self.send_request(indicators, month_first, month_last)
            self.process_data(month_first)
            self.month.loc[diff>=3,'已处理月份'] = month_first.month
        
        
        # 更新上个月
        # 已处理月份为上上个月，则上个月未能处理，处理上个月
        diff = (self.today.month-self.month.loc[:,'已处理月份']).map(lambda x: x+12 if x<0 else x)
        to_update = self.month.loc[diff==2,'代码']
        self.refresh_api_setting()
        if list(to_update):
            indicators = ','.join(list(to_update))
            # 回推一个月，找到一个月前的开始月份
            month_first = datetime.date(self.today.year, self.today.month, 1)
            for i in range(1):
                month_last =  month_first - datetime.timedelta(1)
                month_first = datetime.date(month_last.year, month_last.month, 1)
            log.info('开始更新%s数据（上个月）'%month_first.strftime('%Y-%m'))
            self.send_request(indicators, month_first, month_last)
            self.process_data(month_first)
            self.month.loc[(diff==2),'已处理月份'] = month_first.month
    
    def update_season(self):
        """
        更新频率为季的数据，更新上个季
        """
        to_update = self.season.loc[self.month_season[self.today.month]!=self.season['已处理季度'],'代码']
        self.refresh_api_setting()
        if list(to_update):
            indicators = ','.join(list(to_update))
            # 1,2,3月更新的是上一年的第四季
            if self.month_season[self.today.month] == '04':
                year = self.today.year-1
            else:
                year = self.today.year
            day = datetime.date(year, self.month_season[self.month_season[self.today.month]], 1)
            log.info('开始更新%s年%s季的数据'%(str(year),self.month_season[self.today.month]))
            self.send_request(indicators, day, self.today,"S") 
            self.process_data(day,"S")
    
    def send_request(self,indicators,start,Lstart,MS='M'):
        # 获得数据
        raw_data = w.edb(indicators,start.__format__('%Y/%m/%d'))
        # 提取数据
        # Wind数据返回比较特别，只有一个月的时候会返回一个列表，数据位置和指标ID位置对应；
        # 多个月的时候是二维表，第i个表表示第i个指标ID的值，某些指标返回的时候可能会返回超过所需的数据
        if len(raw_data.Times) == 1:
            self.data = {raw_data.Codes[i]:raw_data.Data[0][i] for i in range(len(raw_data.Codes))}
        elif MS=="S":
            self.data = {raw_data.Codes[i]:raw_data.Data[i][0] for i in range(len(raw_data.Codes))}
        else:
            if Lstart in raw_data.Times:
                self.data = {raw_data.Codes[i]:raw_data.Data[i][raw_data.Times.index(Lstart)] for i in range(len(raw_data.Codes))}
            else:
                self.data = {raw_data.Codes[i]:np.nan for i in range(len(raw_data.Codes))}
    
        
    def process_data(self,start, MS='M'):
        df = copy.deepcopy(self.api_setting)
        df.loc[:,'数值'] = df.loc[:,'代码'].map(lambda x: self.data[x] if x in self.data.keys() else np.nan)
        if df.loc[~(df.loc[:,'数值'].isna()),:].empty:
            return
        
        # 仅处理有数据的行，没有查询成功的均不作处理
        df.loc[~(df.loc[:,'数值'].isna()),'已处理月份'] = int(start.month)
#         df.loc[:,'已处理月份'] = df.loc[:,'频率'].map(lambda x: pass if x=='月度' else ' ')
        df.loc[~(df.loc[:,'数值'].isna()),'已处理季度'] = str(self.month_season[int(self.today.month)])
#         df.loc[:,'已处理季度'] = df.loc[:,'频率'].map(lambda x: pass if x=='季度' else ' ')
        
        to_excel = copy.deepcopy(df.loc[~(df.loc[:,'数值'].isna()),:])
        
        # 生成“唯一名称”和“年、月、季”
        to_excel.loc[:,'唯一名称'] = to_excel.loc[:,'地区']+to_excel.loc[:,'指标构成3']+to_excel.loc[:,'指标构成1']+to_excel.loc[:,'指标构成2']+to_excel.loc[:,'指标构成4']
        to_excel.loc[:,'唯一名称'] = to_excel.loc[:,'唯一名称'].replace('\s+','',regex=True) 
        to_excel.loc[:,'年'] = start.year
        
        to_excel.loc[:,'月'] = to_excel.loc[:,'频率'].map(lambda x:int(start.month) if x=='月度' else ' ')
        to_excel.loc[:,'季'] = to_excel.loc[:,'频率'].map(lambda x:self.month_season[int(self.today.month)] if x=='季度' else ' ')   
        
        to_excel = copy.deepcopy(to_excel.loc[:,["列表","唯一名称","指标构成1","指标构成2","指标构成3","指标构成4","地区","频率","单位","数据来源","年","季","月","数值"]])
        
        # 保存文件，成功发送到数据库后保存进度
        file_name = self.write_result(start, copy.deepcopy(to_excel.loc[:,:]), MS)
        if self.post_data(file_name):
            self.write_progress(copy.deepcopy(df.loc[:,:]))
        return 
        
    def write_progress(self,to_write):
        """
        保存当前的进度
        """
        to_write.drop('数值',inplace=True,axis=1)
        to_write.to_excel(self.api_setting_path,sheet_name="wind API",index=False)
#         book = openpyxl.load_workbook(self.api_setting_path)
#         write = pd.ExcelWriter(self.api_setting_path,engine='openpyxl')
#         write.book = book
#         write.sheets = {ws.title:ws for ws in book.worksheets}
#         to_write.to_excel(write,sheet_name="wind API",index=False)
#         write.save()
#         write.close()
        log.info('进度已保存')
    
    def write_result(self,date,df,MS):
        """
        生成excel数据，并且调用发送数据库程序
        """
        if MS == "M":
            file_name = '%s-wind处理结果.xlsx'%(date.__format__('%Y-%m-%d'))
            df.to_excel(self.output_folder +"\\"+ file_name, engine='openpyxl', index=False)
        else:
            file_name = "%s季-wind处理结果.xlsx"%(self.month_season[self.today.month])
            df.to_excel(self.output_folder +"\\"+ file_name, engine='openpyxl', index=False)   
        log.info("%s已生成,共计查询%d条数据"%(file_name,len(df)))
        return file_name
    
    def post_data(self,file_name):
        """
        发送到数据库
        """
        
        ### ！！！！ 注意：保存的excel文件需要用以下程序调用一次office excel重新保存一次才能够上传到服务器
        xlApp = wc.DispatchEx("Excel.Application")
        xlApp.Visible = 0
        xlApp.DisplayAlerts = 0
        print(self.output_folder +"\\"+ file_name)
        excel = xlApp.Workbooks.Open(self.output_folder +"\\"+ file_name)
        excel.SaveAs(self.output_folder +"\\"+ file_name)
        excel.Close()
        xlApp.Quit()
        
        file = {'file':(file_name,open(self.output_folder +"\\"+ file_name,'rb'),'application/vnd.ms-excel')}
        data = MultipartEncoder(fields=file)
        response = requests.post(url,files=file)
        
        if response.status_code != 200 or response.json()['status'] != '200':
            print(response.json())
            return False
        else:
            log.info('已导入数据库')
            return True

def handle_exception(exc_type,exc_value,exc_trackback):
    if issubclass(exc_type,KeyboardInterrupt):
        sys.__excepthook__(exc_type,exc_value,exc_traceback)
        return
    log.error("Exception!",exc_info=(exc_type,exc_value,exc_traceback))
sys.excepthook = handle_exception


# 程序运行，如果连接失败直接返回 "Connection Error"
w.start()
if w.isconnected() :
    url = "http://192.168.152.52:9000/indicatorValues/importIndicatorValue" # 数据库API
    x = Wind_API(root+'\\API 处理配置文件-汇总.xlsx', root+'\\处理结果') # 配置文件位置，输出结果位置
    x.to_update()
    w.close()
else:
    print("Connection Error")

log.info('程序结束')

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
